## Import necessary library

In [1]:
import os

import boto3
import sagemaker
from sagemaker.sklearn import SKLearn
from sagemaker.local import LocalSession

import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/tochukwunwoke/Library/Application Support/sagemaker/config.yaml


## Specify sagemaker session and role

In [2]:
sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}
account = boto3.client('sts').get_caller_identity().get('Account')

role = "<insert your aws role>"


## Train the model locally

In [4]:
training_input_path = "file:///Users/tochukwunwoke/Documents/projects/sagemaker_local/data/iris_train.csv"
env = {
       "SM_CHANNEL_TRAIN": training_input_path,
       }
local_output_path = 'file:///Users/tochukwunwoke/Documents/projects/sagemaker_local/trained_model'

# training_input_path = "s3://modeldata1234567/train/train.csv"
# remote_output_path = 's3://sagemaker-us-east-2-930627915954/local_model/'

sk_estimator = SKLearn(
    session=sagemaker_local_session,
    entry_point="train.py", 
    role=role,
    job_name = "local_training",
    instance_count=1,
    instance_type='local',
    py_version="py3",
    framework_version="1.2-1",
    hyperparameters={"n_estimators":4},
    output_path = local_output_path,
    environment=env
)
sk_estimator.fit({'train': training_input_path}) 

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-10-05-11-19-50-745
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' found using Docker CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:sagemaker.local.image:Using the long-lived AWS credentials found in session
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-xo2h1:
    command: train
    container_name: fyrhuo1uhr-

Login Succeeded



What's Next?
  View a summary of image vulnerabilities and recommendations → docker scout quickview 510948584623.dkr.ecr.af-south-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3
INFO:sagemaker.local.image:image pulled: 510948584623.dkr.ecr.af-south-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3
INFO:sagemaker.local.image:docker command: docker compose -f /private/var/folders/3l/wrkgmmgd7ygbbzfgtpcwcx640000gn/T/tmpl0r86_xa/docker-compose.yaml up --build --abort-on-container-exit


time="2024-10-05T13:52:06+01:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmpl0r86_xa\".\nSet `external: true` to use an existing network"
 Container fyrhuo1uhr-algo-1-xo2h1  Creating
 algo-1-xo2h1 The requested image's platform (linux/amd64) does not match the detected host platform (linux/arm64/v8) and no specific platform was requested 
 Container fyrhuo1uhr-algo-1-xo2h1  Created
Attaching to fyrhuo1uhr-algo-1-xo2h1
fyrhuo1uhr-algo-1-xo2h1  | 2024-10-05 12:52:08,643 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
fyrhuo1uhr-algo-1-xo2h1  | 2024-10-05 12:52:08,647 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
fyrhuo1uhr-algo-1-xo2h1  | 2024-10-05 12:52:08,658 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
fyrhuo1uhr-algo-1-xo2h1  | 2024-10-05 12:52:08,666 sagemaker-training-toolkit INFO     instance_groups entry no

INFO:root:copying /private/var/folders/3l/wrkgmmgd7ygbbzfgtpcwcx640000gn/T/tmpl0r86_xa/algo-1-xo2h1/output/success -> /private/var/folders/3l/wrkgmmgd7ygbbzfgtpcwcx640000gn/T/tmpl0r86_xa/artifacts/output
INFO:root:creating /private/var/folders/3l/wrkgmmgd7ygbbzfgtpcwcx640000gn/T/tmpl0r86_xa/artifacts/output/data
INFO:root:copying /private/var/folders/3l/wrkgmmgd7ygbbzfgtpcwcx640000gn/T/tmpl0r86_xa/model/model.joblib -> /private/var/folders/3l/wrkgmmgd7ygbbzfgtpcwcx640000gn/T/tmpl0r86_xa/artifacts/model
INFO:root:copying /private/var/folders/3l/wrkgmmgd7ygbbzfgtpcwcx640000gn/T/tmpl0r86_xa/compressed_artifacts/model.tar.gz -> /Users/tochukwunwoke/Documents/projects/sagemaker_local/trained_model
INFO:root:copying /private/var/folders/3l/wrkgmmgd7ygbbzfgtpcwcx640000gn/T/tmpl0r86_xa/compressed_artifacts/output.tar.gz -> /Users/tochukwunwoke/Documents/projects/sagemaker_local/trained_model
INFO:sagemaker.local.image:===== Job Complete =====


 Container fyrhuo1uhr-algo-1-xo2h1  Stopped


## Deploy Model Locally

In [5]:
predictor = sk_estimator.deploy(initial_instance_count=1, instance_type="local")

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-10-05-14-05-47-522
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-10-05-14-05-47-522
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using

Attaching to 8ytef5gcb1-algo-1-jxr4d
8ytef5gcb1-algo-1-jxr4d  | 2024-10-05 14:05:54,889 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
8ytef5gcb1-algo-1-jxr4d  | 2024-10-05 14:05:54,901 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
8ytef5gcb1-algo-1-jxr4d  | 2024-10-05 14:05:54,907 INFO - sagemaker-containers - nginx config: 
8ytef5gcb1-algo-1-jxr4d  | worker_processes auto;
8ytef5gcb1-algo-1-jxr4d  | daemon off;
8ytef5gcb1-algo-1-jxr4d  | pid /tmp/nginx.pid;
8ytef5gcb1-algo-1-jxr4d  | error_log  /dev/stderr;
8ytef5gcb1-algo-1-jxr4d  | 
8ytef5gcb1-algo-1-jxr4d  | worker_rlimit_nofile 4096;
8ytef5gcb1-algo-1-jxr4d  | 
8ytef5gcb1-algo-1-jxr4d  | events {
8ytef5gcb1-algo-1-jxr4d  |   worker_connections 2048;
8ytef5gcb1-algo-1-jxr4d  | }
8ytef5gcb1-algo-1-jxr4d  | 
8ytef5gcb1-algo-1-jxr4d  | http {
8ytef5gcb1-algo-1-jxr4d  |   include /etc/nginx/mime.types;
8ytef5gcb1-algo-1-jxr4d  |   default_type application/octet-stream;
8ytef

INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 10
INFO:sagemaker.local.entities:Container still not up, got: 502


8ytef5gcb1-algo-1-jxr4d  | Processing /opt/ml/code
8ytef5gcb1-algo-1-jxr4d  |   Preparing metadata (setup.py) ... 2024/10/05 14:05:56 [crit] 37#37: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 192.168.65.1, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "localhost:8080"
8ytef5gcb1-algo-1-jxr4d  | 192.168.65.1 - - [05/Oct/2024:14:05:56 +0000] "GET /ping HTTP/1.1" 502 166 "-" "python-urllib3/2.2.3"
8ytef5gcb1-algo-1-jxr4d  | done
8ytef5gcb1-algo-1-jxr4d  | Building wheels for collected packages: train
8ytef5gcb1-algo-1-jxr4d  |   Building wheel for train (setup.py) ... done
8ytef5gcb1-algo-1-jxr4d  |   Created wheel for train: filename=train-1.0.0-py2.py3-none-any.whl size=4301 sha256=b4f46b121a0d83a5606a7354fd4e4609d04b9623849a7d8e8306f9e54f75faf8
8ytef5gcb1-algo-1-jxr4d  |   Stored in directory: /home/model-server/tmp/pip-ephem-wheel-cache-is3tc46b/wheels/f3/75/57/15

INFO:sagemaker.local.entities:Checking if serving container is up, attempt: 15


8ytef5gcb1-algo-1-jxr4d  | 2024-10-05 14:06:01,868 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
8ytef5gcb1-algo-1-jxr4d  | 192.168.65.1 - - [05/Oct/2024:14:06:02 +0000] "GET /ping HTTP/1.1" 200 0 "-" "python-urllib3/2.2.3"
!

## Test the local deployed endpoint

In [6]:
test_data = pd.read_csv("data/iris_test.csv")
test_data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,6.1,2.8,4.7,1.2,1
1,5.7,3.8,1.7,0.3,0
2,7.7,2.6,6.9,2.3,2
3,6.0,2.9,4.5,1.5,1
4,6.8,2.8,4.8,1.4,1


In [7]:
test_df = test_data.drop('label', axis = 1)
pred_values = test_data['label']
test_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,6.1,2.8,4.7,1.2
1,5.7,3.8,1.7,0.3
2,7.7,2.6,6.9,2.3
3,6.0,2.9,4.5,1.5
4,6.8,2.8,4.8,1.4


In [8]:
print(predictor.predict(test_df.values))
print(pred_values.values)

8ytef5gcb1-algo-1-jxr4d  | 2024-10-05 14:07:06,880 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]8ytef5gcb1-algo-1-jxr4d  | 192.168.65.1 - - [05/Oct/2024:14:07:08 +0000] "POST /invocations HTTP/1.1" 200 368 "-" "python-urllib3/2.2.3"

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:28:05 +0000] [114] [CRITICAL] WORKER TIMEOUT (pid:127)
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:28:05 +0000] [114] [CRITICAL] WORKER TIMEOUT (pid:152)
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:28:05 +0000] [114] [CRITICAL] WORKER TIMEOUT (pid:163)
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:28:05 +0000] [152] [INFO] Worker exiting (pid: 152)
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:28:05 +0000] [127] [INFO] Worker exiting (pid: 127)
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:28:05 +0000] [163] [INFO] Worker exiting (pid: 163)
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:28:06 +0000] [280] [INFO] Booting worker with pid: 280
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:28:06 +0000] [281] [INFO] Booting worker with pid: 281
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:28:06 +0000] [282] [INFO] Booting worker with pid: 282
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 14:55:26 +0000] [114] [CRITICAL] WORKER TIMEOUT (pid:140)
8ytef5gcb1-algo-1-jxr4d  | [2024-10-05 